In [8]:
from openai import OpenAI
from openai.types.responses import ParsedResponse
import json
import os
from app.database.entities.cluster_unit_entity import ClusterUnitEntity
from local_host_tester import app


In [11]:
scraper_cluster_id = "68e1300bc04b5f3e7ed60c93"
cluster_unit_entities = app.get(f"/clustering/get_cluster_units?scraper_cluster_id={scraper_cluster_id}").json()["cluster_unit_entities"]
cluster_unit_entities = [ClusterUnitEntity.model_validate(unit) for unit in cluster_unit_entities]
print(f"number of cluster units: {len(cluster_unit_entities)}")

number of cluster units: 5819


In [ ]:
def get_llm_labels(conversation_thread: str, final_reddit_message: str):
    # TODO should we add "Take into account that when we are talking about problems and solution, to use the problem solution framework that is used in startups to come up with new solutions to existing problems"
    # TODO We must explain better what kind of solutions or problems we are talking about. But how do we define a solution and problem correctly?
    # Else we get these kind of results: See below the example message evaluation
    """
    # Message
    Do I call my kid deaf of HoH to other Deaf people?
    Parent to a three year old with moderate &amp; moderately severe hearing levels currently. (His hearing may be progressive? ENT to do some tests)

    We went to our first deaf event about a month ago. It was lovely, people came up to us to talk, signed slow and repeated when we asked, they probably overlooked my grammar errors, and we think two people gave our kid a sign name. (We are just learning ASL so not completely sure)

    The people we spoke with often asked if our kid was deaf and I would sign HE HOH. Afterwards I started wondering if that was the right thing to do? I was originally only thinking of his hearing levels, but after I started thinking more of the cultural aspect. I don't know if my kid knows what either sign/word means or the difference between them. 

    It feels like it probably doesn't matter too much right now but I want to set an example for how he might choose for himself in the future if that makes sense?

    # Final user message
    What his %loss?? If it medium to profoundly deaf that would be deaf  ,,, if it mild to
    Medium that would be HoH
    {
    "rationale": "Asks what the percent hearing loss is and offers a classification rule, so it's both asking for info and proposing how to label.",
    "labels": ["solution_seeking", "solution_proposing"],
    "sentiment": "neutral"
    }"""

    prompt = \
    """
    You are the judge that scores by assigning labels to a reddit message. Return in JSON

    You are supposed to assign a label to the RedditMessage to evaluate. This can be a post, comment or a nested comment. It is critical to evaluate from the persepctive from the user that wrote the RedditMessage to evaluate
    You will receive the conversation thread that the comment or reply has reacted towards. The thread starts with the post title and content, followed by the comment thread, shown in hierarchical structure. The user that wrote the RedditMessage replied to the conversation thread.
    It is critical to assign the label to the final RedditMessage. Take the conversationthread as context when it is of value. 
    
    ### label (choose multiple if applicable)
    problem_description (The use describes a problem that it faces)
    frustration_expression (The user expresser a frustration about something they face)
    solution_seeking (The user is looking for a solution)
    solution_attempted (The user explains an experience to using a solution)
    solution_proposing (The user suggests a solution to someone else)
    agreement_empathy (The user is emphathetic towards another user)
    none_of_the_above

    ### Sentiment choose one
    (negative, neutral, positive)

    ### OUTPUT JSON:
    {
      "rationale": "one short sentence",
      "labels": [...],
      "sentiment": "negative/neutral/postive
    }
    
    ### ConversationThread
    {{conversation_thread}}

    ### RedditMessage to evaluate
    {{final_reddit_message}}
    """\
        .replace("{{conversation_thread}}", conversation_thread)\
        .replace("{{final_reddit_message}}", final_reddit_message)
    llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    model_name ='gpt-5-mini'
    messages = [
      {'role': 'system', 'content': prompt},
      {'role': 'user', 'content': ''} ]
    response = llm.chat.completions.create(
                model=model_name, 
                messages=messages)
    return response



In [32]:
def process_sample_single_cluster(single_cluster_unit: ClusterUnitEntity):
    response = get_llm_labels(str(single_cluster_unit.thread_path_text), single_cluster_unit.text)
    print("# Message:")
    [print(message) for message in single_cluster_unit.thread_path_text]
    print("\n # Final user message")
    print(single_cluster_unit.text)

    print(response.choices[0].message.content)


In [33]:
process_sample_single_cluster(cluster_unit_entities[1500])

# Message:
My experience as a hearing person in a deaf world
As a preface, I am not writing this in an attempt to harm or attack any individual or group. I simply want to shine a light on my somewhat unique experience in the Deaf community as a hearing person.  
  
Background context: I learned ASL in college to fulfill a language credit. The language didn't mean much to me until my second year or so, when I really started to get involved in some extracurriculars, despite the language having nothing to do with the core of my studies. I ended up falling for, and eventually marrying the woman of my dreams whom I met independently of my studies, and who happened to be deaf/fluent in sign.  
  
So my home language shifted to ASL being my primary language. I sign more than I speak now. I became connected to plenty of deaf communities both with her and independent of her. Things pop up from time to time where people vent about hearing people, and I get that, but then other topics come up lik

In [37]:
process_sample_single_cluster(cluster_unit_entities[1500])

# Message:
My experience as a hearing person in a deaf world
As a preface, I am not writing this in an attempt to harm or attack any individual or group. I simply want to shine a light on my somewhat unique experience in the Deaf community as a hearing person.  
  
Background context: I learned ASL in college to fulfill a language credit. The language didn't mean much to me until my second year or so, when I really started to get involved in some extracurriculars, despite the language having nothing to do with the core of my studies. I ended up falling for, and eventually marrying the woman of my dreams whom I met independently of my studies, and who happened to be deaf/fluent in sign.  
  
So my home language shifted to ASL being my primary language. I sign more than I speak now. I became connected to plenty of deaf communities both with her and independent of her. Things pop up from time to time where people vent about hearing people, and I get that, but then other topics come up lik

In [38]:
process_sample_single_cluster(cluster_unit_entities[1500])

# Message:
My experience as a hearing person in a deaf world
As a preface, I am not writing this in an attempt to harm or attack any individual or group. I simply want to shine a light on my somewhat unique experience in the Deaf community as a hearing person.  
  
Background context: I learned ASL in college to fulfill a language credit. The language didn't mean much to me until my second year or so, when I really started to get involved in some extracurriculars, despite the language having nothing to do with the core of my studies. I ended up falling for, and eventually marrying the woman of my dreams whom I met independently of my studies, and who happened to be deaf/fluent in sign.  
  
So my home language shifted to ASL being my primary language. I sign more than I speak now. I became connected to plenty of deaf communities both with her and independent of her. Things pop up from time to time where people vent about hearing people, and I get that, but then other topics come up lik

In [34]:
process_sample_single_cluster(cluster_unit_entities[1600])

# Message:
Quarantine Boredom - My first makeup video!


 # Final user message
Omg the eyeliner was so funny!! Love this video, such a goofy and fun personality and I loved the sign for eyelashes.
{
  "rationale": "This is a positive, supportive compliment referencing the video and personality.",
  "labels": ["agreement_empathy"],
  "sentiment": "positive"
}


In [35]:
process_sample_single_cluster(cluster_unit_entities[1750])

# Message:
How are people still this ignorant?

Hearing people are distracted by sound all the time. We should probably remove that from all White House videos as well.
Captions can get pretty distracting too you know, so those are gone. And 97% of people, a real statistic and not one I pulled out of nowhere, are also distracted by the video and all the movement, so the video footage is also gone.  A blank screen- distractions just aren't worth it.

 # Final user message
I personally love captions. It is because of how many people find caption annoying I am amazed sometimes. Once I went to watch a movie (John Wick) and one of the audience actually went to the backroom/projector room and told those guys to disable captions because it was annoying. My friends were pretty upset about it and they went there for me to argue about it, but I didn't wanted to ruin the day for everyone so I just told them to calm down. But it was pretty much a bad day.
{
  "rationale": "The message expresses a 

In [46]:
cluster_unit_entities[1753].text

'These glasses give me headaches because the captions are on a completely different layer than the movie screen. I hate them'

In [39]:
process_sample_single_cluster(cluster_unit_entities[1750])

# Message:
How are people still this ignorant?

Hearing people are distracted by sound all the time. We should probably remove that from all White House videos as well.
Captions can get pretty distracting too you know, so those are gone. And 97% of people, a real statistic and not one I pulled out of nowhere, are also distracted by the video and all the movement, so the video footage is also gone.  A blank screen- distractions just aren't worth it.

 # Final user message
I personally love captions. It is because of how many people find caption annoying I am amazed sometimes. Once I went to watch a movie (John Wick) and one of the audience actually went to the backroom/projector room and told those guys to disable captions because it was annoying. My friends were pretty upset about it and they went there for me to argue about it, but I didn't wanted to ruin the day for everyone so I just told them to calm down. But it was pretty much a bad day.
{
  "rationale": "The message describes a 

# Message:
How are people still this ignorant?

Hearing people are distracted by sound all the time. We should probably remove that from all White House videos as well.
Captions can get pretty distracting too you know, so those are gone. And 97% of people, a real statistic and not one I pulled out of nowhere, are also distracted by the video and all the movement, so the video footage is also gone.  A blank screen- distractions just aren't worth it.


 # Final user message
I personally love captions. It is because of how many people find caption annoying I am amazed sometimes. Once I went to watch a movie (John Wick) and one of the audience actually went to the backroom/projector room and told those guys to disable captions because it was annoying. My friends were pretty upset about it and they went there for me to argue about it, but I didn't wanted to ruin the day for everyone so I just told them to calm down. But it was pretty much a bad day.


{
  "rationale": "The message describes a positive view of captions and recounts a related incident involving captions.",
  "labels": ["problem_description"],
  "sentiment": "positive"
}


----
The result was given by GPT5-nano. So they may be somewhat skewed because of the small model. The label is correct, the message is indeed a problem description. But the rationale is not about positivity in the final message. Also the sentiment is not correct. The possible reason for problems might come from two modality of the message. The user first says they love captions, which is positive. Then they say they have a problem with people turning captions of. 

In [47]:
process_sample_single_cluster(cluster_unit_entities[1753])

# Message:
How are people still this ignorant?

Hearing people are distracted by sound all the time. We should probably remove that from all White House videos as well.
Captions can get pretty distracting too you know, so those are gone. And 97% of people, a real statistic and not one I pulled out of nowhere, are also distracted by the video and all the movement, so the video footage is also gone.  A blank screen- distractions just aren't worth it.
I personally love captions. It is because of how many people find caption annoying I am amazed sometimes. Once I went to watch a movie (John Wick) and one of the audience actually went to the backroom/projector room and told those guys to disable captions because it was annoying. My friends were pretty upset about it and they went there for me to argue about it, but I didn't wanted to ruin the day for everyone so I just told them to calm down. But it was pretty much a bad day.
most movies offer glasses and doesn't bother anyone..I highly dou

In [ ]:
cluster_unit_entities[1912].

'What his %loss?? If it medium to profoundly deaf that would be deaf  ,,, if it mild to\nMedium that would be HoH'

# Message:
Do I call my kid deaf of HoH to other Deaf people?
Parent to a three year old with moderate &amp; moderately severe hearing levels currently. (His hearing may be progressive? ENT to do some tests)

We went to our first deaf event about a month ago. It was lovely, people came up to us to talk, signed slow and repeated when we asked, they probably overlooked my grammar errors, and we think two people gave our kid a sign name. (We are just learning ASL so not completely sure)

The people we spoke with often asked if our kid was deaf and I would sign HE HOH. Afterwards I started wondering if that was the right thing to do? I was originally only thinking of his hearing levels, but after I started thinking more of the cultural aspect. I don't know if my kid knows what either sign/word means or the difference between them. 

It feels like it probably doesn't matter too much right now but I want to set an example for how he might choose for himself in the future if that makes sense?

 # Final user message
What his %loss?? If it medium to profoundly deaf that would be deaf  ,,, if it mild to
Medium that would be HoH
{
  "rationale": "The message asks for information to classify the child's hearing loss (deaf vs HoH).",
  "labels": ["problem_description", "solution_seeking"],
  "sentiment": "neutral"
}

---
here it went incorrect. As the LLM evaluated the conversation thread instead of the final user message. 

In [52]:
process_sample_single_cluster(cluster_unit_entities[1912])

# Message:
Do I call my kid deaf of HoH to other Deaf people?
Parent to a three year old with moderate &amp; moderately severe hearing levels currently. (His hearing may be progressive? ENT to do some tests)

We went to our first deaf event about a month ago. It was lovely, people came up to us to talk, signed slow and repeated when we asked, they probably overlooked my grammar errors, and we think two people gave our kid a sign name. (We are just learning ASL so not completely sure)

The people we spoke with often asked if our kid was deaf and I would sign HE HOH. Afterwards I started wondering if that was the right thing to do? I was originally only thinking of his hearing levels, but after I started thinking more of the cultural aspect. I don't know if my kid knows what either sign/word means or the difference between them. 

It feels like it probably doesn't matter too much right now but I want to set an example for how he might choose for himself in the future if that makes sense?

In [34]:
response.choices[0].message.content

"{'properties': {'keywords': {'description': 'List of keywords', 'items': {'type': 'string'}, 'title': 'Keywords', 'type': 'array'}}, 'required': ['keywords'], 'title': 'RedditKeywords', 'type': 'object', 'keywords': ['captions accuracy', 'captions sync timing', 'SDH subtitles', 'ASL overlay', 'how to enable captions', 'YouTube captions quality', 'caption customization font size contrast']}"

In [37]:
response_dict = {'properties': {'keywords': {'description': 'List of keywords', 'items': {'type': 'string'}, 'title': 'Keywords', 'type': 'array'}}, 'required': ['keywords'], 'title': 'RedditKeywords', 'type': 'object', 'keywords': ['captions accuracy', 'captions sync timing', 'SDH subtitles', 'ASL overlay', 'how to enable captions', 'YouTube captions quality', 'caption customization font size contrast']}
response_dict

{'properties': {'keywords': {'description': 'List of keywords',
   'items': {'type': 'string'},
   'title': 'Keywords',
   'type': 'array'}},
 'required': ['keywords'],
 'title': 'RedditKeywords',
 'type': 'object',
 'keywords': ['captions accuracy',
  'captions sync timing',
  'SDH subtitles',
  'ASL overlay',
  'how to enable captions',
  'YouTube captions quality',
  'caption customization font size contrast']}

In [20]:
response.choices[0].to_dict()

{'finish_reason': 'stop',
 'index': 0,
 'message': {'content': "{'properties': {'keywords': {'description': 'List of keywords', 'items': {'type': 'string'}, 'title': 'Keywords', 'type': 'array'}}, 'required': ['keywords'], 'title': 'RedditKeywords', 'type': 'object', 'keywords': ['captions accuracy', 'captions sync timing', 'SDH subtitles', 'ASL overlay', 'how to enable captions', 'YouTube captions quality', 'caption customization font size contrast']}",
  'refusal': None,
  'role': 'assistant',
  'annotations': []}}

In [22]:
response.to_dict()

{'id': 'chatcmpl-COhSOaSgPtbGbzLJKjFD6OfCxAkW3',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': "{'properties': {'keywords': {'description': 'List of keywords', 'items': {'type': 'string'}, 'title': 'Keywords', 'type': 'array'}}, 'required': ['keywords'], 'title': 'RedditKeywords', 'type': 'object', 'keywords': ['captions accuracy', 'captions sync timing', 'SDH subtitles', 'ASL overlay', 'how to enable captions', 'YouTube captions quality', 'caption customization font size contrast']}",
    'refusal': None,
    'role': 'assistant',
    'annotations': []}}],
 'created': 1760002956,
 'model': 'gpt-5-nano-2025-08-07',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': None,
 'usage': {'completion_tokens': 2028,
  'prompt_tokens': 665,
  'total_tokens': 2693,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 1920,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_d

In [ ]:
response_1 = llm.chat.completions.create(
            model=model_name, 
            messages=messages,
            reasoning_effort="minimal")
response_1

ChatCompletion(id='chatcmpl-COhpZIHiSmQj8LokQDgqgzLbZbcf9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="{'properties': {'keywords': {'description': 'List of keywords', 'items': {'type': 'string'}, 'title': 'Keywords', 'type': 'array'}}, 'required': ['keywords'], 'title': 'RedditKeywords', 'type': 'object'}", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1760004393, model='gpt-5-nano-2025-08-07', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=65, prompt_tokens=665, total_tokens=730, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [40]:
response_1.choices[0].message.content

"{'properties': {'keywords': {'description': 'List of keywords', 'items': {'type': 'string'}, 'title': 'Keywords', 'type': 'array'}}, 'required': ['keywords'], 'title': 'RedditKeywords', 'type': 'object'}"

In [41]:
resonse_1_text = {'properties': {'keywords': {'description': 'List of keywords', 'items': {'type': 'string'}, 'title': 'Keywords', 'type': 'array'}}, 'required': ['keywords'], 'title': 'RedditKeywords', 'type': 'object'}
resonse_1_text

{'properties': {'keywords': {'description': 'List of keywords',
   'items': {'type': 'string'},
   'title': 'Keywords',
   'type': 'array'}},
 'required': ['keywords'],
 'title': 'RedditKeywords',
 'type': 'object'}

In [42]:
response_2 = llm.chat.completions.create(
            model="gpt-5", 
            messages=messages,
            reasoning_effort="minimal")
response_2

ChatCompletion(id='chatcmpl-COhrOHW8qPVa13ZivUHGNUnwzvpZT', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="{'properties': {'keywords': {'description': 'List of keywords', 'items': {'type': 'string'}, 'title': 'Keywords', 'type': 'array'}}, 'required': ['keywords'], 'title': 'RedditKeywords', 'type': 'object', 'keywords': ['open captions vs closed captions', 'subtitles for the deaf and hard of hearing (SDH)', 'auto captions accuracy YouTube Netflix', 'caption timing out of sync complaint', 'visual sound cues (doorbell siren laughter) in captions', 'ASL overlay picture-in-picture video', 'recommendations captioning tools apps', 'accessibility settings video players (HDR contrast font)'] }", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1760004506, model='gpt-5-2025-08-07', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(compl

In [ ]:
response_2.choices[0].message.content

"{'properties': {'keywords': {'description': 'List of keywords', 'items': {'type': 'string'}, 'title': 'Keywords', 'type': 'array'}}, 'required': ['keywords'], 'title': 'RedditKeywords', 'type': 'object', 'keywords': ['open captions vs closed captions', 'subtitles for the deaf and hard of hearing (SDH)', 'auto captions accuracy YouTube Netflix', 'caption timing out of sync complaint', 'visual sound cues (doorbell siren laughter) in captions', 'ASL overlay picture-in-picture video', 'recommendations captioning tools apps', 'accessibility settings video players (HDR contrast font)'] }"

In [45]:
response_2_dict ={'properties': {'keywords': {'description': 'List of keywords', 'items': {'type': 'string'}, 'title': 'Keywords', 'type': 'array'}}, 'required': ['keywords'], 'title': 'RedditKeywords', 'type': 'object', 'keywords': ['open captions vs closed captions', 'subtitles for the deaf and hard of hearing (SDH)', 'auto captions accuracy YouTube Netflix', 'caption timing out of sync complaint', 'visual sound cues (doorbell siren laughter) in captions', 'ASL overlay picture-in-picture video', 'recommendations captioning tools apps', 'accessibility settings video players (HDR contrast font)'] }
response_2_dict

{'properties': {'keywords': {'description': 'List of keywords',
   'items': {'type': 'string'},
   'title': 'Keywords',
   'type': 'array'}},
 'required': ['keywords'],
 'title': 'RedditKeywords',
 'type': 'object',
 'keywords': ['open captions vs closed captions',
  'subtitles for the deaf and hard of hearing (SDH)',
  'auto captions accuracy YouTube Netflix',
  'caption timing out of sync complaint',
  'visual sound cues (doorbell siren laughter) in captions',
  'ASL overlay picture-in-picture video',
  'recommendations captioning tools apps',
  'accessibility settings video players (HDR contrast font)']}

In [46]:
response_3 = llm.chat.completions.create(
            model="gpt-5", 
            messages=messages,
            reasoning_effort="high")
response_3

ChatCompletion(id='chatcmpl-COhsuKhABXnsS8HFN0VIkPuv3jxRg', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "properties": {\n    "keywords": {\n      "description": "List of keywords",\n      "items": {\n        "type": "string"\n      },\n      "title": "Keywords",\n      "type": "array"\n    }\n  },\n  "required": [\n    "keywords"\n  ],\n  "title": "RedditKeywords",\n  "type": "object",\n  "keywords": [\n    "SDH subtitles",\n    "YouTube auto captions",\n    "Netflix subtitles out of sync",\n    "subtitles too small",\n    "dialogue boost",\n    "Live Caption",\n    "ASL interpreter overlay",\n    "OpenAI Whisper subtitles"\n  ]\n}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1760004600, model='gpt-5-2025-08-07', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=5013, prompt_tokens=665, total_toke

In [50]:
json.loads(response_3.choices[0].message.content)

{'properties': {'keywords': {'description': 'List of keywords',
   'items': {'type': 'string'},
   'title': 'Keywords',
   'type': 'array'}},
 'required': ['keywords'],
 'title': 'RedditKeywords',
 'type': 'object',
 'keywords': ['SDH subtitles',
  'YouTube auto captions',
  'Netflix subtitles out of sync',
  'subtitles too small',
  'dialogue boost',
  'Live Caption',
  'ASL interpreter overlay',
  'OpenAI Whisper subtitles']}

## Review

OpenAI is blowing up the tokens with the reasoning. The documentation also changed since last time I used OPENAI, and when the github was created that I found that does pain point research on reddit automatically. Let's try the responses api instead of the chatcompletions api now

In [4]:
from typing import List
from pydantic import BaseModel, Field


class RedditKeywords(BaseModel): 
    keywords: List[str] = Field(description="List of keywords")

In [ ]:


response_responses_1 = llm.responses.parse(model="gpt-5",
                    input=messages,
                    text_format=RedditKeywords)

In [ ]:
response_responses_1.output_parsed

RedditKeywords(keywords=['SDH subtitles', 'YouTube auto captions accuracy', 'subtitle desync streaming', 'no captions on videos', 'Netflix subtitle readability settings', 'open captions vs closed captions', 'HoH friendly video recommendations', 'ASL interpreter overlay'])

In [56]:
response_responses_2 = llm.responses.parse(model="gpt-5-mini",
                    input=messages,
                    text_format=RedditKeywords)
response_responses_2.output_parsed

RedditKeywords(keywords=['closed captions', 'auto-generated captions', 'SDH subtitles', 'caption sync', 'caption accuracy', 'sign language interpreter', 'how to enable captions', 'download subtitles'])

In [23]:
response_responses_3 = llm.responses.parse(model="gpt-5-nano",
                    input=messages,
                    text_format=RedditKeywords)
response_responses_3.output_parsed

RedditKeywords(keywords=['live captions', 'captions accuracy', 'subtitle readability', 'ASL overlay', 'video accessibility', 'scene-by-scene transcripts', 'lip-reading reliance', 'auto-generated vs human captions'])

In [ ]:
response_responses_3.usage.to_dict()

In [5]:
response_responses_4 = llm.responses.parse(model="gpt-4.1",
                    input=messages,
                    text_format=RedditKeywords)
response_responses_4.output_parsed

RedditKeywords(keywords=['captioning accuracy', 'inaccessible videos', 'auto-generated subtitles issues', 'hard of hearing recommendations', 'sign language integration', 'subtitle customization', 'deaf video experience', 'missing audio cues'])

In [27]:
response_responses_4.usage.output_tokens_details.reasoning_tokens

0

In [32]:
f"{1}.json" == "1.json"

True

In [22]:
if not os.path.exists("data"):
    os.mkdir("data")
os.listdir("data")

['search_result.json',
 'cached_comments1.json',
 'post_entity.json',
 'cached_post1.json',
 'comments_result.json',
 'search_result1.json']

In [59]:
response_responses_5 = llm.responses.parse(model="gpt-oss-120b",
                    input=messages,
                    text_format=RedditKeywords)
response_responses_5.output_parsed

RedditKeywords(keywords=['closed captions accuracy', 'auto-generated subtitles errors', 'best captioning tool for deaf', 'YouTube caption sync issue', 'VTT vs SRT for accessibility', 'how to add captions to personal videos', 'deaf-friendly video platforms', 'caption lag frustration'])

In [63]:
response_responses_5.output_parsed.model_dump()

{'keywords': ['closed captions accuracy',
  'auto-generated subtitles errors',
  'best captioning tool for deaf',
  'YouTube caption sync issue',
  'VTT vs SRT for accessibility',
  'how to add captions to personal videos',
  'deaf-friendly video platforms',
  'caption lag frustration']}

In [64]:
response_responses_5.usage

ResponseUsage(input_tokens=665, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=416, output_tokens_details=OutputTokensDetails(reasoning_tokens=256), total_tokens=1081)